In [40]:
import pandas as pd

df = pd.read_csv('cleaned_data_no_outliers.csv')

df.head()


,Unnamed: 0,rating,reviews,amenities,house_rules,price,country,bathrooms,beds,guests,toilets,bedrooms,studios,checkin_final,checkout_final
0,0,4.710000,64,"Mountain view,Valley view,Lake access,Kitchen,...","Check-in: Flexible,Check out: 12:00 pm,Pets ar...",8078,Turkey,1,1,2,0,2,0,Flexible,12.0
1,1,4.824406,0,"Kitchen,Wifi,Dedicated workspace,Free parking ...","Check-in: 4:00 pm - 11:00 pm,Check out: 10:00 ...",4665,Turkey,2,2,4,0,2,0,16,10.0
2,2,4.850000,68,"Mountain view,Kitchen,Wifi,Dedicated workspace...","Check-in: After 1:00 pm,Check out: 12:00 pm,Se...",5991,Georgia,1,3,4,0,1,0,13,12.0
3,3,5.000000,13,"Mountain view,Valley view,Kitchen,Wifi,Free pa...","Check-in: After 2:00 pm,Check out: 12:00 pm,No...",11339,Turkey,1,2,4,0,1,0,14,12.0
4,4,4.824406,0,"Kitchen,Wifi,Free parking on premises,TV,Air c...","Check-in: After 2:00 pm,Check out: 12:00 pm,No...",6673,Turkey,1,1,2,0,1,0,14,12.0


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11160 entries, 0 to 11159
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      11160 non-null  int64  
 1   rating          11137 non-null  float64
 2   reviews         11160 non-null  object 
 3   amenities       11160 non-null  object 
 4   house_rules     11160 non-null  object 
 5   price           11160 non-null  int64  
 6   country         11160 non-null  object 
 7   bathrooms       11160 non-null  int64  
 8   beds            11160 non-null  int64  
 9   guests          11160 non-null  int64  
 10  toilets         11160 non-null  int64  
 11  bedrooms        11160 non-null  int64  
 12  studios         11160 non-null  int64  
 13  checkin_final   10465 non-null  object 
 14  checkout_final  9078 non-null   float64
dtypes: float64(2), int64(8), object(5)
memory usage: 1.3+ MB


In [42]:
df.drop('Unnamed: 0' , axis=1, inplace=True)

In [84]:
X = df[[ 'bathrooms', 'beds', 'rating' , 'bedrooms', 'studios' , 'guests']]

y = df['price']


In [46]:
df['rating'].isnull().value_counts()

rating
False    11137
True        23
Name: count, dtype: int64

In [48]:
df['bathrooms'].isnull().value_counts()

bathrooms
False    11160
Name: count, dtype: int64

In [49]:
df.dropna(subset=['rating'], inplace=True)

In [51]:
df['checkout_final'].fillna(12, inplace=True)

In [85]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)


In [86]:
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV


model1 = Ridge()
model2 = RandomForestRegressor()
model3 = GradientBoostingRegressor()

param_grid1 = {'alpha': [0.1, 1.0, 10.0]}

grid1 = GridSearchCV(model1, param_grid1, cv=10)

grid1.fit(X_scaled, y)

best_model1 = grid1.best_estimator_

In [87]:
from sklearn.ensemble import VotingRegressor

voting_regressor = VotingRegressor(estimators=[
    ('ridge', best_model1),
    ('random_forest', model2),
    ('gradient_boosting', model3)
])

voting_regressor.fit(X_scaled, y)


VotingRegressor(estimators=[('ridge', Ridge(alpha=10.0)),
                            ('random_forest', RandomForestRegressor()),
                            ('gradient_boosting', GradientBoostingRegressor())])

In [88]:
from sklearn.metrics import mean_squared_error , r2_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.1, random_state=42)

voting_regressor.fit(X_train, y_train)

y_pred = voting_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')



Mean Squared Error: 172756055.44007495


In [89]:
r2 = r2_score(y_test, y_pred)
print(f'R^2 Score: {r2}')

R^2 Score: 0.32463929205544606


In [93]:
adjustment_factor = 0.35
predicted_price = voting_regressor.predict([[1,0,4.8,1,0,1]]) * adjustment_factor

In [94]:
print(round(predicted_price[0]))

8865


Saving Model

In [95]:
import joblib
filename = 'model.sav'
joblib.dump(voting_regressor, filename)

['model.sav']